# Regression (Linear)

**Context:** At this point, you have the building blocks for developing many commonly-used ML models. Using these building blocks, we will develop a commonly used ML model---regression. A regression model is a *predictive* model (i.e. a conditional distribution), in which the variable we're interested in predicting is *continuous*. 

**Challenge:** Recall that in a conditional distribution, the parameters of the distribution depend on what we're conditioning. For example, when predicting the probability of intorixaction, $I$, given the day of the week, $D$, we can use a model like,
\begin{align}
p_{I | D}(i | d) = \mathrm{Ber}(\rho(d)) = \underbrace{\rho(d)^{i} \cdot \left(1 - \rho(d) \right)^{1 - i}}_{\text{Bernoulli PMF (see Wikipedia)}},
\end{align}
where the parameter, $\rho(d)$, is a function of the condition (i.e. the day $d$):
\begin{align} \rho(d) &= \begin{cases}
0.1 & \text{if $d$ is weekday} \\
0.4 & \text{if $d$ is weekend}
\end{cases} 
\end{align}
As in this model, in all models we've considered so far, the parameters depend on the condition using if-else expressions (e.g. if $d$ is a weekday, then $\rho = 0.1$, else $\rho = 0.4$). Unfortunately, as models become more complex, this approach for specifying a model becomes unwieldy. For example, if instead of conditioning on something simple, like the day, we instead conditioned on something complicated, like medical image---$1000 \times 1000$ pixels in size---how would we write the appropriate if-else expression? Out of a total of 1-million pixels in the image, which ones are important to even include in the if-else expression? The classes of predictive models we will introduce now---regression and (later) classification---precisely allow us to circumvent this issue. How? By making the parameter a more interesting function of the condition. Continuing with the above example, we can define, $\rho(d)$ to be a linear function of $d$:
\begin{align}
\rho(d) &= \mathrm{intercept} + \mathrm{slope} \cdot d.
\end{align}
We can similarly make $\rho(d)$ a quadratic, or sinusoidal, or polynomial function of $d$---there are endless possibilities. We will start with simple, linear functions, and build up to more complex functions (using neural networks).

**Outline:**
* Introduce predictive models generally
* Instantiate a regression model as a specific type of predictive model
* Implement the regression in `NumPyro`
* Understand how to quantify model performance

## Predictive Models

Since regression models are a specific type of predictive model, let's first describe what a predictive model is using the toolkit we've developed so far. Specifically, we'll describe a predictive model using the directed graphical model and its corresponding generative process. Then, we will write down its joint data distribution and MLE objective.

**Directed Graphical Model.** Suppose we have a data set of observations of two RVs: "inputs" $X$ and "outputs" $Y$. Our goal is to learn to predict $Y$ from $X$ (i.e. learn a conditional distribution $p_{Y | X}$). A predictive model has the following graphical representation:
<div class="canva-centered-embedding">
  <div class="canva-iframe-container">
    <iframe loading="lazy" class="canva-iframe"
      src="https:&#x2F;&#x2F;www.canva.com&#x2F;design&#x2F;DAGJv58NH1k&#x2F;-MhOv0aEL8dP0kRgjrIN6Q&#x2F;view?embed">
    </iframe>
  </div>
</div>

**Generative Process.** A predictive model has the following corresponding generative process: for $n = 1, \dots, N$,
\begin{align}
x_n &\sim p_X(x_n) \\
y_n | x_n &\sim p_{Y | X}(\cdot | x_n; \theta) 
\end{align}
Notice that we've chosen our marginal over the inputs, $p_X(x_n)$, to *not depend* on any parameters. This is because we *just* care about learning how to predict $Y$ from $X$. That is, we assume our use-case will be that we are *given* specific $x$'s for which we want to make predictions---we therefore don't need to sample them from $p_X(\cdot)$. As you will see in the derivation of our objective function, $p_X(x_n)$ will not impact our MLE in any way. 

**Joint Data Log-Likelihood.** For the above model, we have the following joint data log-likelihood:
\begin{align}
\log p(\mathcal{D}; \theta) &= \log \prod\limits_{n=1}^N p(\mathcal{D}_n; \theta) \quad (\text{i.i.d observations}) \\
&= \sum\limits_{n=1}^N \log p(\mathcal{D}_n; \theta) \quad (\text{since } \log(a \cdot b) = \log a + \log b) \\
&= \sum\limits_{n=1}^N \log p_{X, Y}(x_n, y_n; \theta) \quad (\text{since } \mathcal{D}_n = (x_n, y_n)) \\
&= \sum\limits_{n=1}^N \log p_{Y | X}(y_n | x_n; \theta) + \log p_X(x_n) \quad (\text{using our factorization of the joint}) 
\end{align}

**MLE Objective.** Our goal is to maximize the probability of the joint data log-likelihood with respect to parameters $\theta$:
\begin{align}
\mathrm{argmax}_\theta \log p(\mathcal{D}; \theta) &= \mathrm{argmax}_\theta \sum\limits_{n=1}^N \log p_{Y | X}(y_n | x_n; \theta) + \log p_X(x_n) \quad (\text{substituting in our joint data log-likelihood}) \\
&= \mathrm{argmin}_\theta -\sum\limits_{n=1}^N \left( \log p_{Y | X}(y_n | x_n; \theta) + \log p_X(x_n) \right) \quad (\text{taking an argmax is like taking an argmin of the negative}) \\
&= \mathrm{argmin}_\theta -\sum\limits_{n=1}^N \log p_{Y | X}(y_n | x_n; \theta) - \underbrace{\sum\limits_{n=1}^N \log p_X(x_n)}_{\text{does not depend on } \theta} \quad (\text{we split out the sum into two parts}) \\
&= \mathrm{argmin}_\theta -\sum\limits_{n=1}^N \log p_{Y | X}(y_n | x_n; \theta) \quad (\text{term that doesn't depend on $\theta$ doesn't affect argmin}) 
\end{align}
This gives us the following objective (or loss function):
\begin{align}
\mathcal{L}(\theta) = -\sum\limits_{n=1}^N \log p_{Y | X}(y_n | x_n; \theta)
\end{align}
Now that we have our objective, the only thing missing now is a specific choice of $p_{Y | X}(y_n | x_n; \theta)$. We will next select $p_{Y | X}(y_n | x_n; \theta)$ to give us a regression model. Later, we will do the same for other models (classification). 

## The Regression Model

**The Model.** Looking at our objective function, $\mathcal{L}(\theta)$, the only thing we need to define is $p_{Y | X}(y_n | x_n; \theta)$. In a regression model, we define it as follows:
\begin{align}
p_{Y | X}(\cdot | x_n; \underbrace{W, \sigma}_{\theta}) = \mathcal{N}\left( \mu(x_n; W), \sigma^2 \right),
\end{align}
where $\mu(\cdot; W)$ is a function parameterized by $W$. By this, we mean that the shape of the function is determined by the choice of $W$. 

Putting this all together, our generative process says:
1. We're given an input, $x_n$, from our data.
2. We feed it through our function $\mu(\cdot; W)$ to predict what $y_n$ will be, on average.
3. Finally, we add some Gaussian noise to the result. This Gaussian noise represents the error in our measurement. For example, our devices that measure height, blood pressure, etc. at the IHH are correct on average, but are always off by some small amount of random noise. How much noise is determining by the variance of the Gaussian, $\sigma^2$.

**Choosing a functional form.** So what is $\mu(\cdot; W)$ exactly? For now we will start simple, but a bit later in the course, we will build up to very fancy models, in which $\mu$ is represented by an expressive model, like a neural network. So let's pick $\mu(\cdot; W)$ to be a linear function (i.e. a line with a slope and intercept).

Let our inputs, $x_n$, be $D$-dimensional. We access the $d$-th dimension of $x_n$ using the notation, $x_n^{(d)}$. For example $x_n^{(1)}$ can represent the patient's height, $x_n^{(2)}$ can represent the patient's blood pressure, etc. We then define $\mu(\cdot; W)$ as follows:
\begin{align}
\mu(x_n; W) &= W^{(0)} + W^{(1)} \cdot x_n^{(1)} + W^{(2)} \cdot x_n^{(2)} + \dots + W^{(D)} \cdot x_n^{(D)} \\
&= W^{(0)} + \sum\limits_{d=1}^D W^{(d)} \cdot x_n^{(d)}
\end{align}
Here, $W$ has $D + 1$ dimensions; $W^{(0)}$ is the intercept, and $W^{(1)}, \dots, W^{(D)}$ are the slopes.